In [1]:
using Dates
using JuMP
using CSV
using DataFrames
using LinearAlgebra

In [2]:
ENV["COLUMNS"] = 200;

In [27]:
using COVIDResourceAllocation

In [4]:
states = ["CT", "DE", "MA", "MD", "ME", "NH", "NJ", "NY", "PA", "RI", "VT"]
start_date = Date(2020, 4, 1)
end_date   = Date(2020, 5, 1)
travel_threshold_hours = 4.0;

In [5]:
pct_nurses_available = 0.5
patients_per_nurse_covid = 2.5
nurse_hrs_per_week_covid = 36

@show nurse_hrs_per_day_covid = nurse_hrs_per_week_covid / 7
@show nurses_days_per_day_covid = 24 / nurse_hrs_per_day_covid

@show nurse_days_per_patient_day_covid = nurses_days_per_day_covid / patients_per_nurse_covid;

nurse_hrs_per_day_covid = nurse_hrs_per_week_covid / 7 = 5.142857142857143
nurses_days_per_day_covid = 24 / nurse_hrs_per_day_covid = 4.666666666666666
nurse_days_per_patient_day_covid = nurses_days_per_day_covid / patients_per_nurse_covid = 1.8666666666666665


In [6]:
adj = adjacencies(states, level=:state, source=:google, threshold=travel_threshold_hours);

In [7]:
forecast_active = forecast(
    states, start_date, end_date,
    level=:state,
    source=:ihme,
    forecast_type=:active,
    patient_type=:regular,
    bound_type=:mean,
);

In [8]:
demand = forecast_active * Float32(nurse_days_per_patient_day_covid);

In [9]:
nurses = n_nurses(states) * Float32(pct_nurses_available);

In [10]:
_nurses_beds = n_nurses(states, source=:beds);
_nurses_empl = n_nurses(states, source=:employment);
_nurses_ahrf = n_nurses(states, source=:ahrf);
DataFrame(
    state = states,
    from_beds = _nurses_beds,
    from_empl = _nurses_empl,
    from_ahrf = _nurses_ahrf,
)

,state,from_beds,from_empl,from_ahrf
,String,Float32,Float32,Float32
1,CT,2251.67,12120.0,4593.0
2,DE,609.0,3234.06,1322.0
3,MA,4428.67,38220.5,10146.0
4,MD,2851.33,29581.0,6169.0
5,ME,639.333,2346.56,1962.0
6,NH,641.667,1484.33,1934.0
7,NJ,3999.33,71498.5,6652.0
8,NY,9991.33,83342.8,19547.0
9,PA,8227.33,65427.3,14189.0


In [19]:
model = reusable_resource_allocation(
    nurses,
    zeros(Float32, size(demand)...),
    demand,
    adj,
    send_new_only=false,
    sendrecieve_switch_time=0,
    min_send_amt=0,
    smoothness_penalty=0,
    setup_cost=0,
    sent_penalty=0.01,
    verbose=true
)
sent = value.(model[:sent])
println("termination status: ", termination_status(model))
println("solve time: ", round(solve_time(model), digits=3), "s")
println("objective function value: ", round(objective_value(model), digits=3))

Academic license - for non-commercial use only
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (mac64)
Optimize a model with 4492 rows, 4092 columns and 218023 nonzeros
Model fingerprint: 0xceb99501
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-02, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+00, 3e+04]

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolve removed 3812 rows and 1893 columns
Presolve time: 0.04s
Presolved: 680 rows, 2199 columns, 119365 nonzeros

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.389e+05
 Factor NZ  : 2.039e+05 (roughly 3 MBytes of memory)
 Factor Ops : 8.093e+07 (less than 1 second per iteration)
 Threads    : 4

Barrier performed 0 iterations in 0.09 seconds
Barrier solve interrupted - model solved by another algorithm


Solved with dual simplex
Solved in 878 iterations and 0.10 seconds
Optimal 

In [20]:
results = NurseAllocationResults.results_all(sent, nurses, demand, states, start_date);

In [21]:
println("Total sent: ", results.total_sent)
println("Total shortage: ", results.total_shortage)
println("Average load: ", results.average_load)

Total sent: 22355.767333984375
Total shortage: 751284.4462547302
Average load: 2.200377453344581


In [22]:
results.summary_table

,state,total_sent,total_received,initial_nurses,total_nurse_days,total_demand,total_shortage,average_load
,String,Float64,Float64,Float32,Float64,Float32,Float64,Float64
1,CT,1130.1,682.549,2296.5,57521.1,137636.0,81047.7,2.39893
2,DE,555.203,6.76872,661.0,3482.77,9455.91,5988.4,2.71157
3,MA,4215.34,329.833,5073.0,36738.6,114486.0,78193.4,3.11158
4,MD,1958.43,0.0,3084.5,34908.2,99014.6,64561.2,2.83643
5,ME,893.438,0.0,981.0,2758.74,3644.35,945.27,1.33326
6,NH,878.932,0.0,967.0,2730.1,5710.92,3009.01,2.09184
7,NJ,0.0,10245.6,3326.0,361801.0,428721.0,68866.2,1.20337
8,NY,5266.21,9848.77,9773.5,502966.0,833241.0,330976.0,1.64552
9,PA,5720.86,196.782,7094.5,48486.4,153955.0,105879.0,3.16583


In [23]:
results.sent_matrix_table

,state,CT,DE,MA,MD,ME,NH,NJ,NY,PA,RI,VT
,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,CT,0.0,6.76872,0.0,0.0,0.0,0.0,926.547,0.0,196.782,0.0,0.0
2,DE,0.0,0.0,0.0,0.0,0.0,0.0,555.203,0.0,0.0,0.0,0.0
3,MA,0.0,0.0,0.0,0.0,0.0,0.0,87.4216,4127.92,0.0,0.0,0.0
4,MD,0.0,0.0,0.0,0.0,0.0,0.0,1958.43,0.0,0.0,0.0,0.0
5,ME,0.0,0.0,44.3132,0.0,0.0,0.0,0.0,0.0,0.0,849.125,0.0
6,NH,682.549,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,196.383,0.0
7,NJ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,NY,0.0,0.0,0.0,0.0,0.0,0.0,5266.21,0.0,0.0,0.0,0.0
9,PA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5720.86,0.0,0.0,0.0


In [24]:
println("First day:")
filter(row -> row.date == start_date, results.complete_table)

First day:


,state,date,sent,received,initial_nurses,current_nurses,demand,shortage,load,sent_to,sent_from
,String,Date,Float64,Float64,Float32,Float64,Float32,Float64,Float64,Any,Any
1,CT,2020-04-01,926.547,682.549,2296.5,2052.5,1369.95,0.0,0.667455,"[(""NJ"", 926.547)]","[(""NH"", 682.549)]"
2,DE,2020-04-01,555.203,0.0,661.0,105.797,97.3051,0.0,0.919735,"[(""NJ"", 555.203)]",[]
3,MA,2020-04-01,4215.34,270.472,5073.0,1128.13,857.661,0.0,0.760248,"[(""NJ"", 87.4216), (""NY"", 4127.92)]","[(""VT"", 270.472)]"
4,MD,2020-04-01,1958.43,0.0,3084.5,1126.07,826.577,0.0,0.734037,"[(""NJ"", 1958.43)]",[]
5,ME,2020-04-01,849.125,0.0,981.0,131.875,75.3669,0.0,0.571502,"[(""RI"", 849.125)]",[]
6,NH,2020-04-01,878.932,0.0,967.0,88.0676,69.2593,0.0,0.786434,"[(""CT"", 682.549), (""RI"", 196.383)]",[]
7,NJ,2020-04-01,0.0,3990.95,3326.0,7316.95,6358.89,0.0,0.869063,[],"[(""CT"", 926.547), (""DE"", 555.203), (""MA"", 87.4216), (""MD"", 1958.43), (""RI"", 463.349)]"
8,NY,2020-04-01,0.0,9848.77,9773.5,19622.3,26053.0,6430.69,1.32772,[],"[(""MA"", 4127.92), (""PA"", 5720.86)]"
9,PA,2020-04-01,5720.86,0.0,7094.5,1373.64,1173.37,0.0,0.854206,"[(""NY"", 5720.86)]",[]


In [25]:
s = "NY"
filter(row -> row.state == s, results.complete_table)

,state,date,sent,received,initial_nurses,current_nurses,demand,shortage,load,sent_to,sent_from
,String,Date,Float64,Float64,Float32,Float64,Float32,Float64,Float64,Any,Any
1,NY,2020-04-01,0.0,9848.77,9773.5,19622.3,26053.0,6430.69,1.32772,[],"[(""MA"", 4127.92), (""PA"", 5720.86)]"
2,NY,2020-04-02,0.0,0.0,9773.5,19622.3,28789.4,9167.12,1.46718,[],[]
3,NY,2020-04-03,0.0,0.0,9773.5,19622.3,31295.0,11672.7,1.59487,[],[]
4,NY,2020-04-04,0.0,0.0,9773.5,19622.3,33469.4,13847.1,1.70568,[],[]
5,NY,2020-04-05,0.0,0.0,9773.5,19622.3,35215.8,15593.5,1.79468,[],[]
6,NY,2020-04-06,0.0,0.0,9773.5,19622.3,36478.0,16855.8,1.85901,[],[]
7,NY,2020-04-07,0.0,0.0,9773.5,19622.3,37250.7,17628.4,1.89839,[],[]
8,NY,2020-04-08,0.0,0.0,9773.5,19622.3,37534.1,17911.9,1.91283,[],[]
9,NY,2020-04-09,0.0,0.0,9773.5,19622.3,37353.7,17731.4,1.90364,[],[]


In [26]:
results.sent_to

Dict{String,Array{String,1}} with 11 entries:
  "RI" => ["NJ"]
  "NJ" => String[]
  "DE" => ["NJ"]
  "MD" => ["NJ"]
  "NH" => ["CT", "RI"]
  "NY" => ["NJ"]
  "ME" => ["MA", "RI"]
  "CT" => ["DE", "NJ", "PA"]
  "MA" => ["NJ", "NY"]
  "PA" => ["NY"]
  "VT" => ["MA"]